<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

In [3]:
data_train = pd.read_csv("C:/Users/pierr/OneDrive/Documents/Projet/Projet_python/dvf_ensae_sbra/good_data/donnees_model/donnees_train.csv")
data_test = pd.read_csv("C:/Users/pierr/OneDrive/Documents/Projet/Projet_python/dvf_ensae_sbra/good_data/donnees_model/donnees_test.csv")
data_valid = pd.read_csv("C:/Users/pierr/OneDrive/Documents/Projet/Projet_python/dvf_ensae_sbra/good_data/donnees_model/donnees_validation.csv")

In [4]:
data_train.drop(["idmutation", "Ind", "sbati_squa", "datemut", "Unnamed: 0"], axis='columns', inplace=True)
data_test.drop(["idmutation", "Ind", "sbati_squa", "datemut", "Unnamed: 0"], axis='columns', inplace=True)
data_valid.drop(["idmutation", "Ind", "sbati_squa", "datemut", "Unnamed: 0"], axis='columns', inplace=True)

In [11]:
def rename_p_t_n(df, dico):
    df.rename(columns= dico, inplace= True)
    return df

 

rename_dico= {"valfoncact2":"valfoncact"}
data_train= rename_p_t_n(data_train, rename_dico)
data_test= rename_p_t_n(data_test, rename_dico)
data_valid= rename_p_t_n(data_valid, rename_dico)

In [15]:
data_train = data_train[data_train["valfoncact"] > 10000]
data_train = data_train[data_train["valfoncact"] < 3000000]
data_test = data_test[data_test["valfoncact"] > 10000]
data_test = data_test[data_test["valfoncact"] < 3000000]
data_valid = data_valid[data_valid["valfoncact"] > 10000]
data_valid = data_valid[data_valid["valfoncact"] < 3000000]

In [16]:
Y = data_train.pop("valfoncact")

In [17]:
sc_X = StandardScaler()
col = [i for i in data_train.columns]
data_train[col] = sc_X.fit_transform(data_train[col])

In [20]:
lasso= LassoCV()
lasso.fit(data_train, Y)
sf_lasso= SelectFromModel(lasso, prefit= True)

In [21]:
selected_variables= data_train.columns[(sf_lasso.get_support())]
print(len(selected_variables))
print(selected_variables)

33
Index(['nblot', 'nbpar', 'nblocmut', 'sbati', 'pp', 'Men', 'Men_pauv',
       'Men_1ind', 'Men_prop', 'Men_fmp', 'Ind_snv', 'Log_av45', 'Log_45_70',
       'Log_70_90', 'Log_soc', 'Ind_0_3', 'Ind_4_5', 'Ind_18_24', 'Ind_25_39',
       'Ind_40_54', 'Ind_65_79', 'Ind_inc', 'ind_par_zo', 'nv_par_hab',
       'ind_par__1', 'THEATRE', 'arrondissement', 'AUTRES_ALIM', 'AUTRES_SERV',
       'CULTURE', 'ENS_PRI', 'MEDECIN', 'POLICE'],
      dtype='object')


In [22]:
sc_Y = StandardScaler()
Y = sc_Y.fit_transform(np.asarray(Y).reshape(-1,1))

In [ ]:
regressor = SVR(kernel='rbf')
svr = regressor.fit(data_train[selected_variables],Y)

C:\Users\pierr\anaconda3\envs\py37\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [ ]:
Yfit = svr.predict(data_train[selected_variables])

In [ ]:
Y = sc_Y.inverse_transform(Y)
Yfit = sc_Y.inverse_transform(Yfit)

In [ ]:
Yfit

In [ ]:
score = svr.score(data_train[selected_variables], Y)
print("R-squared:", score)


In [ ]:
mse = mean_squared_error(Y, Yfit)
rmse = mse**(1/2)

In [ ]:
print(rmse)

In [161]:
Ytest = data_test.pop("valfoncact")

In [162]:
data_test[col] = sc_X.fit_transform(data_test[col])
Ytest = sc_Y.fit_transform(np.asarray(Ytest).reshape(-1,1))

In [164]:
Yfit_test = svr.predict(data_test[selected_variables])

In [167]:
Ytest = sc_Y.inverse_transform(Ytest)
Yfit_test = sc_Y.inverse_transform(Yfit_test)

In [168]:
mse_test = mean_squared_error(Ytest, Yfit)
rmse_test = mse_test**(1/2)

In [169]:
print(rmse_test)

318146.51498972945
